(hover_dir_pad_1, hover_dir_pad_2, hover_numeric_pad)

(5 options, 5 options, 11 options) ~ 275

5 possible actions 

total 1375 state, action combinations

initial state is (A, A, A)
let's say we have to type 456A

(A, A, A)
(A, A, 4) A
(A, A, 5) A
(A, A, 6) A
(A, A, A) A


what if we're in like

(<, ^, 5) and we hit >
(A, ^, 5) and we hit >


(<, ^, 5) and we hit A?

(A, ^, 5) and we hit A?
(^, A, 5) and we hit A?
(^, >, A) and we hit A?

(A, A, 5) and we hit A?
(^, A, A) and we hit A?

(A, A, A) and we hit A?

In [1]:
# AAAAAAAAAAA0
# AAAAAAAAAAA9

In [2]:
import numpy as np

In [3]:
# input = """029A
# 980A
# 179A
# 456A
# 379A"""

input = open("inputs/21").read()

parsed_input = [list(s) for s in input.split()]

In [4]:
parsed_input

[['1', '4', '0', 'A'],
 ['1', '7', '0', 'A'],
 ['1', '6', '9', 'A'],
 ['8', '0', '3', 'A'],
 ['1', '2', '9', 'A']]

In [5]:
keypresses = parsed_input[0]
keypresses

['1', '4', '0', 'A']

In [6]:
import itertools

directional_board = np.array(
    [
        [None, "^", "A"],
        ["<", "v", ">"],
    ]
)

numeric_board = np.array(
    [
        ["7", "8", "9"],
        ["4", "5", "6"],
        ["1", "2", "3"],
        [None, "0", "A"],
    ]
)

dirs = {
    "^": (-1, 0),
    ">": (0, 1),
    "v": (1, 0),
    "<": (0, -1),
}


def in_bounds(tuple, shape):
    return 0 <= tuple[0] < shape[0] and 0 <= tuple[1] < shape[1]


def add_tuple(t1, t2):
    return tuple(map(sum, zip(t1, t2)))


def make_transition_dict(board):
    d = {}

    for i, j in np.ndindex(board.shape):
        if board[i, j] is None:
            continue

        for dir in dirs:
            new_pos = add_tuple((i, j), dirs[dir])
            d[(board[i, j], dir)] = (
                board[new_pos] if in_bounds(new_pos, board.shape) else None
            )

    return d


def manhattan_distances(array_2d: np.ndarray) -> dict:
    """
    Given a 2D NumPy array, returns a dictionary where the keys are tuples
    of two non-None elements, and the values are the Manhattan distance
    between their positions in the array.

    Each pair is represented in both orders, e.g., (A, B) and (B, A).
    """
    non_none_indices = np.argwhere(array_2d != None)
    values = array_2d[non_none_indices[:, 0], non_none_indices[:, 1]]

    elements = list(zip(values, non_none_indices[:, 0], non_none_indices[:, 1]))

    distance_dict = {}
    for (val1, row1, col1), (val2, row2, col2) in itertools.combinations(elements, 2):
        distance = abs(row1 - row2) + abs(col1 - col2)
        distance_dict[(val1, val2)] = distance
        distance_dict[(val2, val1)] = distance  # Include both directions

    return distance_dict


numeric_transition_dict = make_transition_dict(numeric_board)
directional_transition_dict = make_transition_dict(directional_board)
numeric_ds = manhattan_distances(numeric_board)
directional_ds = manhattan_distances(directional_board)

In [7]:
def get_next_state(state, action):
    *dir_hovers, num_hover = state
    dir_hovers_with_action = [action] + dir_hovers

    first_non_A_idx = next(
        (i for i, x in enumerate(dir_hovers_with_action) if x != "A"), None
    )
    # print(first_non_A_idx)

    # print(dir_hovers_with_action, first_non_A_idx)

    if first_non_A_idx is None:
        # print("emitting")
        # in this case we're emitting
        return state, num_hover

    if first_non_A_idx == len(dir_hovers_with_action) - 1:
        # print("modify num hover")
        # we modify the num_hover state
        new_num_hover = numeric_transition_dict[num_hover, dir_hovers_with_action[-1]]

        if not new_num_hover:
            return None

        return (*dir_hovers, new_num_hover), None
    else:
        # print("modify dir hover")
        # we modify the dir_hovers state
        new_dir_hovers_with_action_state = dir_hovers_with_action.copy()

        new_dir_state = directional_transition_dict[
            new_dir_hovers_with_action_state[first_non_A_idx + 1],
            new_dir_hovers_with_action_state[first_non_A_idx],
        ]

        if not new_dir_state:
            return None

        new_dir_hovers_with_action_state[first_non_A_idx + 1] = new_dir_state
        return (*new_dir_hovers_with_action_state[1:], num_hover), None

In [8]:
def get_possible_neighbors(state):
    for action in list(dirs.keys()) + ["A"]:
        if result := get_next_state(state, action):
            # state, emit = result
            # print(action, result)
            yield action, result

In [9]:
from collections import deque


def get_path_length_between_states(start_state, end_state):
    """bfs"""
    to_visit = deque([(start_state, 0)])
    visited = set()

    while to_visit:
        current_node, distance = to_visit.popleft()

        if current_node == end_state:
            return distance

        if current_node in visited:
            continue

        for n, _emit in get_possible_neighbors(current_node):
            to_visit.append((n, distance + 1))

        visited.add(current_node)

    return None


def get_path_between_states(start_state, end_state):
    """
    Performs BFS to find shortest path between states.
    Returns tuple of (path_length, path) if path exists, else (None, None)
    """
    to_visit = deque([(start_state, 0, [(None, None, start_state)])])
    visited = set()

    while to_visit:
        current_node, distance, path = to_visit.popleft()

        if current_node == end_state:
            return distance, path

        if current_node in visited:
            continue

        for action, (n, emit) in get_possible_neighbors(current_node):
            to_visit.append((n, distance + 1, path + [(action, current_node, n)]))

        visited.add(current_node)

    return None, None


In [10]:
# num_robo_hover_states = 2
# num_robo_hover_states = 3
# num_robo_hover_states = 4
# num_robo_hover_states = 5
num_robo_hover_states = 6
# num_robo_hover_states = 7
# num_robo_hover_states = 25

In [11]:
start_robos = tuple("A" for _ in range(num_robo_hover_states))
start_robos

('A', 'A', 'A', 'A', 'A', 'A')

In [12]:
get_path_between_states(("A", "3"), ("A", "5"))

(8,
 [(None, None, ('A', '3')),
  ('v', ('A', '3'), ('>', '3')),
  ('<', ('>', '3'), ('v', '3')),
  ('<', ('v', '3'), ('<', '3')),
  ('A', ('<', '3'), ('<', '2')),
  ('>', ('<', '2'), ('v', '2')),
  ('^', ('v', '2'), ('^', '2')),
  ('A', ('^', '2'), ('^', '5')),
  ('>', ('^', '5'), ('A', '5'))])

In [13]:
get_path_between_states(("A", "A", "3"), ("A", "A", "5"))

(20,
 [(None, None, ('A', 'A', '3')),
  ('v', ('A', 'A', '3'), ('>', 'A', '3')),
  ('<', ('>', 'A', '3'), ('v', 'A', '3')),
  ('A', ('v', 'A', '3'), ('v', '>', '3')),
  ('<', ('v', '>', '3'), ('<', '>', '3')),
  ('A', ('<', '>', '3'), ('<', 'v', '3')),
  ('A', ('<', 'v', '3'), ('<', '<', '3')),
  ('>', ('<', '<', '3'), ('v', '<', '3')),
  ('^', ('v', '<', '3'), ('^', '<', '3')),
  ('>', ('^', '<', '3'), ('A', '<', '3')),
  ('A', ('A', '<', '3'), ('A', '<', '2')),
  ('v', ('A', '<', '2'), ('>', '<', '2')),
  ('A', ('>', '<', '2'), ('>', 'v', '2')),
  ('^', ('>', 'v', '2'), ('A', 'v', '2')),
  ('<', ('A', 'v', '2'), ('^', 'v', '2')),
  ('A', ('^', 'v', '2'), ('^', '^', '2')),
  ('>', ('^', '^', '2'), ('A', '^', '2')),
  ('A', ('A', '^', '2'), ('A', '^', '5')),
  ('v', ('A', '^', '5'), ('>', '^', '5')),
  ('A', ('>', '^', '5'), ('>', 'A', '5')),
  ('^', ('>', 'A', '5'), ('A', 'A', '5'))])

In [14]:
get_path_between_states(("A", "A", "A", "3"), ("A", "A", "A", "5"))

(54,
 [(None, None, ('A', 'A', 'A', '3')),
  ('v', ('A', 'A', 'A', '3'), ('>', 'A', 'A', '3')),
  ('<', ('>', 'A', 'A', '3'), ('v', 'A', 'A', '3')),
  ('<', ('v', 'A', 'A', '3'), ('<', 'A', 'A', '3')),
  ('A', ('<', 'A', 'A', '3'), ('<', '^', 'A', '3')),
  ('>', ('<', '^', 'A', '3'), ('v', '^', 'A', '3')),
  ('A', ('v', '^', 'A', '3'), ('v', 'v', 'A', '3')),
  ('^', ('v', 'v', 'A', '3'), ('^', 'v', 'A', '3')),
  ('>', ('^', 'v', 'A', '3'), ('A', 'v', 'A', '3')),
  ('A', ('A', 'v', 'A', '3'), ('A', 'v', '>', '3')),
  ('v', ('A', 'v', '>', '3'), ('>', 'v', '>', '3')),
  ('<', ('>', 'v', '>', '3'), ('v', 'v', '>', '3')),
  ('<', ('v', 'v', '>', '3'), ('<', 'v', '>', '3')),
  ('A', ('<', 'v', '>', '3'), ('<', '<', '>', '3')),
  ('>', ('<', '<', '>', '3'), ('v', '<', '>', '3')),
  ('^', ('v', '<', '>', '3'), ('^', '<', '>', '3')),
  ('>', ('^', '<', '>', '3'), ('A', '<', '>', '3')),
  ('A', ('A', '<', '>', '3'), ('A', '<', 'v', '3')),
  ('A', ('A', '<', 'v', '3'), ('A', '<', '<', '3')),
  (

if we're at 


A, A, A, 3

and need to get to

A, A, A, 5


let's say we have chosen a path on the numeric pad which goes from 3 -> 6 -> 5 e.g. up then left

in that case we know we need to pass thru

so in total we know we need to go thru

A, A, A, 3
..
A, A, ^, 6
...
A, A, <, 5
..
A, A, A, 5

if instead we go 3 -> 2 -> 5 we have

A, A, A, 3
..
A, A, <, 2
...
A, A, ^, 5
..
A, A, A, 5




A, A, <, _
A, A, ^, _


(A, A, <) -> (A, A, ^)

(A, >, <)



AAAAAAAAA<


AAAAAAAA><
??

AAAAAAAAA>



AAAAAAAAA


AAAAAAAA<

AAA< + 

recurse(len-1) + recurse(len-1)

base case is going to be with 1 robo hover state in which case the distance is just the distance on the directional pad

which we're going to need anyway for the recursion when we sum the recursive calls. i.e. we need the actual paths between the different keys on the directional pad
from which we can easily compute the distances

In [15]:
import itertools

blah = list(dirs.keys()) + ["A"]

actions_directional_paths = {}

for i, j in itertools.product(blah, blah):
    d, p = get_path_between_states((i, "A"), (j, "A"))
    # print(i, j, d, p)
    # s = "".join([t[-1][0] for t in p])
    actions = [t[0] for t in p[1:]]
    # print(actions)

    actions_directional_paths[i, j] = actions

In [16]:
actions_directional_paths[">", "<"], actions_directional_paths[">", ">"]

(['<', '<'], [])

At our level we know our start and end that we need. We can find the directional sequence we need to achieve that.

We can then recurse at the lower level with the assumption that the lower level is starting at A and needs to end at A. We have to add one for each action in the path (for the activation A)

e.g.

AAAA 
to
AAA>
to
AAA<
to
AAAA

we can find the path from > to v and sum over the recursive calls at a level below


wait when you recurse down do you actually have to start with A and end with A every time?

In [36]:
new_actions_directional_paths = {k: [v] for k, v in actions_directional_paths.items()}
new_actions_directional_paths 

{('^', '^'): [[]],
 ('^', '>'): [['>', 'v']],
 ('^', 'v'): [['v']],
 ('^', '<'): [['v', '<']],
 ('^', 'A'): [['>']],
 ('>', '^'): [['^', '<']],
 ('>', '>'): [[]],
 ('>', 'v'): [['<']],
 ('>', '<'): [['<', '<']],
 ('>', 'A'): [['^']],
 ('v', '^'): [['^']],
 ('v', '>'): [['>']],
 ('v', 'v'): [[]],
 ('v', '<'): [['<']],
 ('v', 'A'): [['^', '>']],
 ('<', '^'): [['>', '^']],
 ('<', '>'): [['>', '>']],
 ('<', 'v'): [['>']],
 ('<', '<'): [[]],
 ('<', 'A'): [['>', '^', '>']],
 ('A', '^'): [['<']],
 ('A', '>'): [['v']],
 ('A', 'v'): [['v', '<']],
 ('A', '<'): [['v', '<', '<']],
 ('A', 'A'): [[]]}

In [17]:
def aaa_directional_distance(num_as, dir_p1, dir_p2):
    actions_path = actions_directional_paths[dir_p1, dir_p2]
    path = ["A"] + actions_path + ["A"]

    if num_as == 0:
        return actions_path

    if num_as == -1:
        return []

    rs = []
    actions = []
    for iter_num, (i, j) in enumerate(zip(path, path[1:]), start=1):
        # at the base level where there are no As we don't need to start and end our actions with A
        r = aaa_directional_distance(num_as - 1, i, j)
        rs.append(((i, j), r))

        actions.extend(r)
        if iter_num == len(path) - 1:
            # we don't add it in this case
            pass
        else:
            # actions.append(j)
            actions.append("A")

    print(f"Looking at action path {path} at level {num_as} for {dir_p1} to {dir_p2}")
    print(f"Path sums: {rs}")
    print(f"Our actions were {actions}")
    print(
        f"At level {num_as} we have {len(actions)} steps for {dir_p1} to {dir_p2} with addtl total of {len(path) - 2}"
    )
    print()

    return actions

In [18]:
aaa_directional_distance(3, ">", "<")

Looking at action path ['A', 'v', '<', 'A'] at level 1 for A to v
Path sums: [(('A', 'v'), ['v', '<']), (('v', '<'), ['<']), (('<', 'A'), ['>', '^', '>'])]
Our actions were ['v', '<', 'A', '<', 'A', '>', '^', '>']
At level 1 we have 8 steps for A to v with addtl total of 2

Looking at action path ['A', '<', 'A'] at level 1 for v to <
Path sums: [(('A', '<'), ['v', '<', '<']), (('<', 'A'), ['>', '^', '>'])]
Our actions were ['v', '<', '<', 'A', '>', '^', '>']
At level 1 we have 7 steps for v to < with addtl total of 1

Looking at action path ['A', 'A'] at level 1 for < to <
Path sums: [(('A', 'A'), [])]
Our actions were []
At level 1 we have 0 steps for < to < with addtl total of 0

Looking at action path ['A', '>', '^', '>', 'A'] at level 1 for < to A
Path sums: [(('A', '>'), ['v']), (('>', '^'), ['^', '<']), (('^', '>'), ['>', 'v']), (('>', 'A'), ['^'])]
Our actions were ['v', 'A', '^', '<', 'A', '>', 'v', 'A', '^']
At level 1 we have 9 steps for < to A with addtl total of 3

Looking 

['v',
 '<',
 'A',
 '<',
 'A',
 '>',
 '^',
 '>',
 'A',
 'v',
 '<',
 '<',
 'A',
 '>',
 '^',
 '>',
 'A',
 'A',
 'v',
 'A',
 '^',
 '<',
 'A',
 '>',
 'v',
 'A',
 '^',
 'A',
 'A',
 'v',
 '<',
 'A',
 '^',
 '>',
 'A',
 '<',
 'A',
 'v',
 '<',
 'A',
 '>',
 '^',
 '>',
 'A',
 'v',
 'A',
 '<',
 'A',
 '^',
 '>',
 'A',
 '<',
 'A',
 '>']

In [19]:
aaa_directional_distance(0, ">", "<")

['<', '<']

In [20]:
aaa_directional_distance(1, ">", "<")

Looking at action path ['A', '<', '<', 'A'] at level 1 for > to <
Path sums: [(('A', '<'), ['v', '<', '<']), (('<', '<'), []), (('<', 'A'), ['>', '^', '>'])]
Our actions were ['v', '<', '<', 'A', 'A', '>', '^', '>']
At level 1 we have 8 steps for > to < with addtl total of 2



['v', '<', '<', 'A', 'A', '>', '^', '>']

In [21]:
get_path_between_states(("A", ">", "0"), ("A", "<", "0"))

(8,
 [(None, None, ('A', '>', '0')),
  ('v', ('A', '>', '0'), ('>', '>', '0')),
  ('<', ('>', '>', '0'), ('v', '>', '0')),
  ('<', ('v', '>', '0'), ('<', '>', '0')),
  ('A', ('<', '>', '0'), ('<', 'v', '0')),
  ('A', ('<', 'v', '0'), ('<', '<', '0')),
  ('>', ('<', '<', '0'), ('v', '<', '0')),
  ('^', ('v', '<', '0'), ('^', '<', '0')),
  ('>', ('^', '<', '0'), ('A', '<', '0'))])

In [22]:
aaa_directional_distance(2, ">", "<")

Looking at action path ['A', 'v', '<', '<', 'A'] at level 1 for A to <
Path sums: [(('A', 'v'), ['v', '<']), (('v', '<'), ['<']), (('<', '<'), []), (('<', 'A'), ['>', '^', '>'])]
Our actions were ['v', '<', 'A', '<', 'A', 'A', '>', '^', '>']
At level 1 we have 9 steps for A to < with addtl total of 3

Looking at action path ['A', 'A'] at level 1 for < to <
Path sums: [(('A', 'A'), [])]
Our actions were []
At level 1 we have 0 steps for < to < with addtl total of 0

Looking at action path ['A', '>', '^', '>', 'A'] at level 1 for < to A
Path sums: [(('A', '>'), ['v']), (('>', '^'), ['^', '<']), (('^', '>'), ['>', 'v']), (('>', 'A'), ['^'])]
Our actions were ['v', 'A', '^', '<', 'A', '>', 'v', 'A', '^']
At level 1 we have 9 steps for < to A with addtl total of 3

Looking at action path ['A', '<', '<', 'A'] at level 2 for > to <
Path sums: [(('A', '<'), ['v', '<', 'A', '<', 'A', 'A', '>', '^', '>']), (('<', '<'), []), (('<', 'A'), ['v', 'A', '^', '<', 'A', '>', 'v', 'A', '^'])]
Our actions

['v',
 '<',
 'A',
 '<',
 'A',
 'A',
 '>',
 '^',
 '>',
 'A',
 'A',
 'v',
 'A',
 '^',
 '<',
 'A',
 '>',
 'v',
 'A',
 '^']

In [33]:
start = ("A", "<", ">", "0")
# actions = ['A', 'A', 'v', 'A', 'A', '^', '<', 'A', '>']
actions = ["A", "A", "v", "A", "^", "<", "A", ">", "v", "A", "^"]

current_state = start
for a in actions:
    r = get_next_state(current_state, a)
    if r:
        current_state, _ = r
        print(a, current_state)
    else:
        print("!")

A ('A', '<', 'v', '0')
A ('A', '<', '<', '0')
v ('>', '<', '<', '0')
A ('>', 'v', '<', '0')
^ ('A', 'v', '<', '0')
< ('^', 'v', '<', '0')
A ('^', '^', '<', '0')
> ('A', '^', '<', '0')
v ('>', '^', '<', '0')
A ('>', 'A', '<', '0')
^ ('A', 'A', '<', '0')


In [34]:
start = ("A", "<", ">", "0")
actions = ["A", "A", "v", "A", "A", "^", "<", "A", ">"]
# actions = ["A", "A", "v", "A", "^", "<", "A", ">", "v", "A", "^"]

current_state = start
for a in actions:
    r = get_next_state(current_state, a)
    if r:
        current_state, _ = r
        print(a, current_state)
    else:
        print("!")

A ('A', '<', 'v', '0')
A ('A', '<', '<', '0')
v ('>', '<', '<', '0')
A ('>', 'v', '<', '0')
A ('>', '>', '<', '0')
^ ('A', '>', '<', '0')
< ('^', '>', '<', '0')
A ('^', 'A', '<', '0')
> ('A', 'A', '<', '0')


In [24]:
get_path_between_states(("A", "A", ">", "0"), ("A", "A", "<", "0"))

(18,
 [(None, None, ('A', 'A', '>', '0')),
  ('v', ('A', 'A', '>', '0'), ('>', 'A', '>', '0')),
  ('<', ('>', 'A', '>', '0'), ('v', 'A', '>', '0')),
  ('A', ('v', 'A', '>', '0'), ('v', '>', '>', '0')),
  ('<', ('v', '>', '>', '0'), ('<', '>', '>', '0')),
  ('A', ('<', '>', '>', '0'), ('<', 'v', '>', '0')),
  ('A', ('<', 'v', '>', '0'), ('<', '<', '>', '0')),
  ('>', ('<', '<', '>', '0'), ('v', '<', '>', '0')),
  ('^', ('v', '<', '>', '0'), ('^', '<', '>', '0')),
  ('>', ('^', '<', '>', '0'), ('A', '<', '>', '0')),
  ('A', ('A', '<', '>', '0'), ('A', '<', 'v', '0')),
  ('A', ('A', '<', 'v', '0'), ('A', '<', '<', '0')),
  ('v', ('A', '<', '<', '0'), ('>', '<', '<', '0')),
  ('A', ('>', '<', '<', '0'), ('>', 'v', '<', '0')),
  ('A', ('>', 'v', '<', '0'), ('>', '>', '<', '0')),
  ('^', ('>', '>', '<', '0'), ('A', '>', '<', '0')),
  ('<', ('A', '>', '<', '0'), ('^', '>', '<', '0')),
  ('A', ('^', '>', '<', '0'), ('^', 'A', '<', '0')),
  ('>', ('^', 'A', '<', '0'), ('A', 'A', '<', '0'))])

In [25]:
actions_directional_paths[">", "<"]

['<', '<']

In [26]:
get_path_between_states((">", "0"), ("<", "0"))

(2,
 [(None, None, ('>', '0')),
  ('<', ('>', '0'), ('v', '0')),
  ('<', ('v', '0'), ('<', '0'))])

In [27]:
get_path_between_states(("A", ">", "0"), ("A", "<", "0"))

(8,
 [(None, None, ('A', '>', '0')),
  ('v', ('A', '>', '0'), ('>', '>', '0')),
  ('<', ('>', '>', '0'), ('v', '>', '0')),
  ('<', ('v', '>', '0'), ('<', '>', '0')),
  ('A', ('<', '>', '0'), ('<', 'v', '0')),
  ('A', ('<', 'v', '0'), ('<', '<', '0')),
  ('>', ('<', '<', '0'), ('v', '<', '0')),
  ('^', ('v', '<', '0'), ('^', '<', '0')),
  ('>', ('^', '<', '0'), ('A', '<', '0'))])

In [28]:
get_path_between_states(("A", "A", "A", ">", "0"), ("A", "A", "A", "<", "0"))

(46,
 [(None, None, ('A', 'A', 'A', '>', '0')),
  ('v', ('A', 'A', 'A', '>', '0'), ('>', 'A', 'A', '>', '0')),
  ('<', ('>', 'A', 'A', '>', '0'), ('v', 'A', 'A', '>', '0')),
  ('<', ('v', 'A', 'A', '>', '0'), ('<', 'A', 'A', '>', '0')),
  ('A', ('<', 'A', 'A', '>', '0'), ('<', '^', 'A', '>', '0')),
  ('>', ('<', '^', 'A', '>', '0'), ('v', '^', 'A', '>', '0')),
  ('A', ('v', '^', 'A', '>', '0'), ('v', 'v', 'A', '>', '0')),
  ('^', ('v', 'v', 'A', '>', '0'), ('^', 'v', 'A', '>', '0')),
  ('>', ('^', 'v', 'A', '>', '0'), ('A', 'v', 'A', '>', '0')),
  ('A', ('A', 'v', 'A', '>', '0'), ('A', 'v', '>', '>', '0')),
  ('v', ('A', 'v', '>', '>', '0'), ('>', 'v', '>', '>', '0')),
  ('<', ('>', 'v', '>', '>', '0'), ('v', 'v', '>', '>', '0')),
  ('<', ('v', 'v', '>', '>', '0'), ('<', 'v', '>', '>', '0')),
  ('A', ('<', 'v', '>', '>', '0'), ('<', '<', '>', '>', '0')),
  ('>', ('<', '<', '>', '>', '0'), ('v', '<', '>', '>', '0')),
  ('^', ('v', '<', '>', '>', '0'), ('^', '<', '>', '>', '0')),
  ('>',

In [29]:
get_path_between_states(("A", "A", "A", "A", ">", "0"), ("A", "A", "A", "A", "<", "0"))


(108,
 [(None, None, ('A', 'A', 'A', 'A', '>', '0')),
  ('v', ('A', 'A', 'A', 'A', '>', '0'), ('>', 'A', 'A', 'A', '>', '0')),
  ('<', ('>', 'A', 'A', 'A', '>', '0'), ('v', 'A', 'A', 'A', '>', '0')),
  ('A', ('v', 'A', 'A', 'A', '>', '0'), ('v', '>', 'A', 'A', '>', '0')),
  ('<', ('v', '>', 'A', 'A', '>', '0'), ('<', '>', 'A', 'A', '>', '0')),
  ('A', ('<', '>', 'A', 'A', '>', '0'), ('<', 'v', 'A', 'A', '>', '0')),
  ('A', ('<', 'v', 'A', 'A', '>', '0'), ('<', '<', 'A', 'A', '>', '0')),
  ('>', ('<', '<', 'A', 'A', '>', '0'), ('v', '<', 'A', 'A', '>', '0')),
  ('^', ('v', '<', 'A', 'A', '>', '0'), ('^', '<', 'A', 'A', '>', '0')),
  ('>', ('^', '<', 'A', 'A', '>', '0'), ('A', '<', 'A', 'A', '>', '0')),
  ('A', ('A', '<', 'A', 'A', '>', '0'), ('A', '<', '^', 'A', '>', '0')),
  ('v', ('A', '<', '^', 'A', '>', '0'), ('>', '<', '^', 'A', '>', '0')),
  ('A', ('>', '<', '^', 'A', '>', '0'), ('>', 'v', '^', 'A', '>', '0')),
  ('^', ('>', 'v', '^', 'A', '>', '0'), ('A', 'v', '^', 'A', '>', '0')